# Exercise session 3, Exercise 2

This is a template for using the ASP solver [clingo](https://potassco.org/clingo/) to work out Exercise 2 for *Exercise session 3*.

First, install clingo, e.g., using: `conda install -c potassco clingo`

To use clingo in Python, import the clingo package.

In [1]:
import clingo

## Exercise 2

### Setting things up
In the answer set program, we will use atoms of the form `finish(Person, Pos)` to represent whether `Person` finished in position `Pos`.

Let's start by declaring some facts about which persons and which positions there are, using `person/1` and `position/1`.

In [2]:
asp_program = """
    % Declare the six persons
    person(alex).
    person(blake).
    person(charlie).
    person(dakota).
    person(emerson).
    person(frankie).
    
    % Declare the six positions
    position(1..6).
"""

Next, as a starting point, we generate all different ways in which they could have finished.

The first step to do this is to make a binary choice for whether any given person finished in any given position.

In [3]:
asp_program += """
    finish(Person,Pos) :- not didnt_finish(Person,Pos), person(Person), position(Pos).
    didnt_finish(Person,Pos) :- not finish(Person,Pos), person(Person), position(Pos).
"""

This does not yet give us a program whose answer sets correspond to the different orderings.

For example, we need to express that every person finished in some position.

In [4]:
asp_program += """
    finish_in_some_position(Person) :- person(Person), finish(Person,_).
    :- not finish_in_some_position(Person), person(Person).
"""

Since we are ultimately only interested in who finished in which position, we declare that we only want to see `finish/2` in the answer sets.

In [5]:
asp_program += """
    #show finish/2.
"""

### Finish the program
The answer set program is not finished yet. Complete the program by adding more facts/rules.

For example, you should make sure that no two people finished in the same position.

And you should express the knowledge that is given about the order in which they finished.

In [6]:
# SOLUTION: finish the program
asp_program += """
    % A person cannot finish in two different positions
    :- person(Person), finish(Person,Pos1), finish(Person,Pos2), Pos1 != Pos2.
    
    % No two different persons can finish in the same position
    :- position(Pos), finish(Person1,Pos), finish(Person2,Pos), Person1 != Person2.
    
    % Dakota didn't finish last
    :- finish(dakota,6).
    
    % Alex, Blake and Frankie were all faster than both Charlie and Dakota
    :- finish(alex,Pos1), finish(charlie,Pos2), Pos1 > Pos2.
    :- finish(blake,Pos1), finish(charlie,Pos2), Pos1 > Pos2.
    :- finish(frankie,Pos1), finish(charlie,Pos2), Pos1 > Pos2.
    :- finish(alex,Pos1), finish(dakota,Pos2), Pos1 > Pos2.
    :- finish(blake,Pos1), finish(dakota,Pos2), Pos1 > Pos2.
    :- finish(frankie,Pos1), finish(dakota,Pos2), Pos1 > Pos2.
    
    % Alex finished in first or second place
    alex_in_top_two :- finish(alex,1).
    alex_in_top_two :- finish(alex,2).
    :- not alex_in_top_two.
    
    % Blake didn't finish in third place
    :- finish(blake,3).
    
    % Frankie came in three spots ahead of Charlie
    frankie_three_ahead_of_charlie :- finish(frankie,Pos1), finish(charlie,Pos2), Pos1 = Pos2 - 3.
    :- not frankie_three_ahead_of_charlie.
"""

### Finding the answer sets

We can use clingo's Python API to find one or more answer sets.

Here is a convenient function to print a given number of answer sets for a given program.

In [7]:
def print_answer_sets(program, num_answer_sets=0):
    
    # Load the answer set program, and call the grounder
    control = clingo.Control(arguments=["--project"])
    control.add("base", [], program)
    control.ground([("base", [])])
    
    # Define a function that will be called when an answer set is found
    # This function sorts the answer set alphabetically, and prints it
    def on_model(model):
        sorted_model = [str(atom) for atom in model.symbols(shown=True)]
        sorted_model.sort()
        print("Answer set: {{{}}}\n".format(", ".join(sorted_model)))
    
    # Ask clingo to find some number of models
    # (using an upper bound of 0 gives all models)
    control.configuration.solve.models = num_answer_sets
    
    # Call the clingo solver, passing on the function on_model for when an answer set is found
    answer = control.solve(on_model=on_model)
    
    # Print a message when no answer set was found
    if answer.satisfiable == False:
        print("No answer sets")

For example, we can print three of the answer sets of our program as follows:

In [8]:
print_answer_sets(asp_program)

Answer set: {finish(alex,2), finish(blake,1), finish(charlie,6), finish(dakota,4), finish(emerson,5), finish(frankie,3)}

Answer set: {finish(alex,2), finish(blake,4), finish(charlie,6), finish(dakota,5), finish(emerson,1), finish(frankie,3)}

Answer set: {finish(alex,2), finish(blake,1), finish(charlie,6), finish(dakota,5), finish(emerson,4), finish(frankie,3)}

Answer set: {finish(alex,1), finish(blake,4), finish(charlie,6), finish(dakota,5), finish(emerson,2), finish(frankie,3)}

Answer set: {finish(alex,1), finish(blake,2), finish(charlie,6), finish(dakota,4), finish(emerson,5), finish(frankie,3)}

Answer set: {finish(alex,1), finish(blake,2), finish(charlie,6), finish(dakota,5), finish(emerson,4), finish(frankie,3)}



Actually, we can also transform the answer sets to some other representation. For example, let's turn them into a list of pairs, that we can easily work with.

In [9]:
def extract_solutions(program, num_solutions=0):
    
    # Load the answer set program, and call the grounder
    control = clingo.Control(arguments=["--project"])
    control.add("base", [], program)
    control.ground([("base", [])])
    
    # This function transforms the answer set into a list of pairs
    def answer_set_to_list(model):
        output = []
        for atom in model.symbols(shown=True):
            if atom.name == "finish":
                output.append((
                    atom.arguments[0].name,
                    atom.arguments[1].number
                ))
        return output
                
    # Ask clingo to find all models (using an upper bound of 0 gives all models)
    control.configuration.solve.models = num_solutions
    
    # Call the clingo solver, transform each answer
    return [answer_set_to_list(answer) for answer in control.solve(yield_=True)]  

In [10]:
for solution in extract_solutions(asp_program):
    print(f"{solution}\n")

[('blake', 1), ('alex', 2), ('frankie', 3), ('dakota', 4), ('emerson', 5), ('charlie', 6)]

[('emerson', 1), ('alex', 2), ('frankie', 3), ('blake', 4), ('dakota', 5), ('charlie', 6)]

[('blake', 1), ('alex', 2), ('frankie', 3), ('emerson', 4), ('dakota', 5), ('charlie', 6)]

[('alex', 1), ('emerson', 2), ('frankie', 3), ('blake', 4), ('dakota', 5), ('charlie', 6)]

[('alex', 1), ('blake', 2), ('frankie', 3), ('dakota', 4), ('emerson', 5), ('charlie', 6)]

[('alex', 1), ('blake', 2), ('frankie', 3), ('emerson', 4), ('dakota', 5), ('charlie', 6)]



Or we can simply check if an answer set program has at least one answer set.

In [11]:
def has_answer_set(program):
    control = clingo.Control(arguments=["--project"])
    control.add("base", [], program)
    control.ground([("base", [])])
        
    control.configuration.solve.models = 1
    answer = control.solve()
    return answer.satisfiable

### Finding answers to the questions

After having represented the knowledge in ASP, let's now turn to answering the three (reasoning) questions about it:
- Who finished in third place?
- In what places could Dakota have finished?
- Can we conclude that exactly one of Blake and Emerson finished in the top two?

Since our ASP program—encoding the given knowledge—only has 6 answer sets, one way of finding the answers to these questions is by looking at the answer sets, and checking the relevant properties in each of them. For example, all answer sets of the program contain the atom `finish(frankie,3)`, so we can conclude that Frankie must have finished in third position.

The following approach shows how we can answer our questions also in the case where the number of answer sets is too large to all inspect.

In [12]:
# SOLUTION: use clingo to find the answers to the questions

# Who finished in third place?
for person in ['alex', 'blake', 'charlie', 'dakota', 'emerson', 'frankie']:
    new_asp_program = asp_program
    new_asp_program += f"""
        :- not finish({person},3).
    """
    if has_answer_set(new_asp_program):
        print(f"{person.capitalize()} could have finished in third position.")

Frankie could have finished in third position.


In [13]:
# In what positions could Dakota have finished?
for position in range(1,7):
    new_asp_program = asp_program
    new_asp_program += f"""
        :- not finish(dakota,{position}).
    """
    if has_answer_set(new_asp_program):
        print(f"Dakota could have finished in position {position}.")

Dakota could have finished in position 4.
Dakota could have finished in position 5.


In [14]:
# Is it possible that not exactly one of Blake and Emerson finished in the top two?
new_asp_program = asp_program
new_asp_program += f"""
    none_of_EB_in_top_two :- finish(blake,B), B > 2, finish(emerson,E), E > 2.
    both_of_EB_in_top_two :- finish(blake,B), B <= 2, finish(emerson,E), E <= 2.
    one_of_EB_in_top_two :- not none_of_EB_in_top_two, not both_of_EB_in_top_two.
    :- one_of_EB_in_top_two.
"""
if has_answer_set(new_asp_program):
    print(f"It is possible that not exactly one of Blake and Emerson finished in the top two.")
else:
    print(f"Exactly one of Blake and Emerson must have finished in the top two.")

Exactly one of Blake and Emerson must have finished in the top two.
